In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import re
from rapidfuzz import process, fuzz
from tqdm import tqdm
from collections import Counter
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\omidg\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
maxmara_df = pd.read_json('maxmara_us_maxmara_us_20250121092247.jl', lines=True)
netaporter_df = pd.read_json('netaporter_us_netaporter_us_20250121050520.jl', lines=True)
luisaviaroma_df = pd.read_json('luisaviaroma_us_luisaviaroma_us_20250121080220.jl', lines=True)
print("Max Mara Data:")
display(maxmara_df.head(1))
print(maxmara_df.info())
print(maxmara_df.isnull().sum())
print("\nNet-A-Porter Data:")
display(netaporter_df.head(1))
print(netaporter_df.info())
print(netaporter_df.isnull().sum())
print("\nLuisa Via Roma Data:")
display(luisaviaroma_df.head(1))
print(luisaviaroma_df.info())
print(luisaviaroma_df.isnull().sum())

Max Mara Data:


,meta,url,product_id,source,country,lang,ref,sex,main_title,images,details,description,color_snippets,outlet_color,excluweb_color,full_path,category_urls,full_ref,labels_identifier,price_hierarchy,detected_sex,clean_url,tags
0,{'upload': '2025-01-21 10:53:27.478076'},{'en': ['https://us.maxmara.com/p-108106510600...,https://us.maxmara.com/p-1081065106001-oggeri-...,Maxmara,[us],en,1081065106,women,{'en': 'Cropped Teddy Bear Icon Coat in alpaca...,{'WHITE': ['https://b2c-media.maxmara.com/sys-...,"{'colors': ['WHITE'], 'material': 'Jersey fabr...",{'en': 'Cropped jacket in the iconic Teddy fab...,{'WHITE': 'https://b2c-media.maxmara.com/sys-m...,{'WHITE': False},{'WHITE': False},"[[Homepage, Coats and Jackets, Coats and Jacke...",[https://us.maxmara.com/coats-and-jackets/wome...,1081065106001,1081065106001,"{'type': 'color', 'WHITE': {'price': {'USD': '...",women,/p-1081065106001-oggeri-white,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3646 entries, 0 to 3645
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   meta               3646 non-null   object
 1   url                3646 non-null   object
 2   product_id         3646 non-null   object
 3   source             3646 non-null   object
 4   country            3646 non-null   object
 5   lang               3646 non-null   object
 6   ref                3646 non-null   int64 
 7   sex                3646 non-null   object
 8   main_title         3646 non-null   object
 9   images             3646 non-null   object
 10  details            3646 non-null   object
 11  description        3646 non-null   object
 12  color_snippets     3646 non-null   object
 13  outlet_color       3646 non-null   object
 14  excluweb_color     3646 non-null   object
 15  full_path          3646 non-null   object
 16  category_urls      3646 non-null   object


,meta,url,product_id,source,country,lang,ref,sex,main_title,images,details,description,outlet_color,excluweb_color,full_path,category_urls,price_hierarchy,detected_sex,clean_url
0,{'upload': '2025-01-21 09:52:49.177951'},{'en': ['https://www.net-a-porter.com/en-us/sh...,https://www.net-a-porter.com/en-us/shop/produc...,Netaporter,[us],en,1647597332512685,women,{'en': 'Le 37 leather-trimmed raffia bucket bag'},{'Beige': ['https://www.net-a-porter.com/varia...,"{'colors': ['Beige'], 'material': 'Raffia, bro...",{'en': 'SAINT LAURENT's 'Le 37' tote is the la...,{'Beige': False},{'Beige': False},[[SAINT LAURENT]],[https://www.net-a-porter.com/en-us/shop/desig...,"{'type': 'color', 'Beige': {'price': {'USD': '...",women,/en-us/shop/product/saint-laurent/bags/bucket-...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7791 entries, 0 to 7790
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   meta             7791 non-null   object
 1   url              7791 non-null   object
 2   product_id       7791 non-null   object
 3   source           7791 non-null   object
 4   country          7791 non-null   object
 5   lang             7791 non-null   object
 6   ref              7791 non-null   int64 
 7   sex              7791 non-null   object
 8   main_title       7791 non-null   object
 9   images           7791 non-null   object
 10  details          7791 non-null   object
 11  description      7791 non-null   object
 12  outlet_color     7791 non-null   object
 13  excluweb_color   7791 non-null   object
 14  full_path        7791 non-null   object
 15  category_urls    7791 non-null   object
 16  price_hierarchy  7791 non-null   object
 17  detected_sex     7791 non-null   

,meta,url,product_id,source,country,lang,ref,sex,main_title,images,details,description,outlet_color,excluweb_color,full_path,category_urls,price_hierarchy,detected_sex
0,{'upload': '2025-01-21 08:03:34.400252'},{'en': ['https://www.luisaviaroma.com/en-us/p/...,https://www.luisaviaroma.com/en-us/p/courreges...,Luisaviaroma,[us],en,80I-YZ0023,women,{'en': 'Rib knit viscose tank top'},{'Black_9999': ['https://images.lvrcdn.com/c/2...,"{'colors': ['Black_9999'], 'material': '70% Vi...",{'en': 'Fixed spaghetti straps Front logo deta...,{'Black_9999': False},{'Black_9999': False},"[[women, COURREGES]]",[https://www.luisaviaroma.com/en-us/shop/women...,"{'type': 'color', 'Black_9999': {'price': {'US...",women


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1445 entries, 0 to 1444
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   meta             1445 non-null   object
 1   url              1445 non-null   object
 2   product_id       1445 non-null   object
 3   source           1445 non-null   object
 4   country          1445 non-null   object
 5   lang             1445 non-null   object
 6   ref              1445 non-null   object
 7   sex              1445 non-null   object
 8   main_title       1445 non-null   object
 9   images           1445 non-null   object
 10  details          1445 non-null   object
 11  description      1445 non-null   object
 12  outlet_color     1445 non-null   object
 13  excluweb_color   1445 non-null   object
 14  full_path        1445 non-null   object
 15  category_urls    1445 non-null   object
 16  price_hierarchy  1445 non-null   object
 17  detected_sex     1417 non-null   

In [3]:
def clean_price(price):
    if price == "" or price is None:
        return np.nan 
    return float(price)

def expand_dataframe(df):
    expanded_rows = []
    for _, row in df.iterrows():
        colors = row['details']['colors']
        for color in colors:
            expanded_row = {
                #'meta': row.get('url', None),
                'url': row.get('url', None),
                'product_id': row.get('product_id', None),
                #'source': row.get('source', None),
                #'country': row.get('country', None),
                #'lang': row.get('lang', None),
                'ref': row.get('ref', None),
                #'sex': row.get('sex', None),
                'main_title': row.get('main_title', None),
                'description': row.get('description', None),
                'full_path': row.get('full_path', None),
                #'category_urls': row.get('category_urls', None),
                #'full_ref': row.get('full_ref', None),
                #'labels_identifier': row.get('labels_identifier', None),
                #'detected_sex': row.get('detected_sex', None),
                #'clean_url': row.get('clean_url', None),
                'details': row.get('details', None),
                'color': color,
                'current_price': clean_price(row.get('price_hierarchy', {}).get(color, {}).get('price', {}).get('USD', None)),
                'previous_price': clean_price(row.get('price_hierarchy', {}).get(color, {}).get('previous_price', {}).get('USD', None)),
                'available_sizes': row.get('details', {}).get('sizes', {}).get(color, None),
                'all_sizes': row.get('details', {}).get('all_sizes', {}).get(color, None),
                'images': row.get('images', {}).get(color, None),
                #'color_snippets': row.get('color_snippets', {}).get(color, None),
                'outlet_color': row.get('outlet_color', {}).get(color, None),
                'excluweb_color': row.get('excluweb_color', {}).get(color, None),
            }
            expanded_rows.append(expanded_row)
    expanded_df = pd.DataFrame(expanded_rows)
    return expanded_df

def product_id_extractor(url, pattern):
    patterns = {
        "max_mara": r"p-(\d+)-",  # Matches "p-<product_id>-"
        "net_a_porter": r"/product/.*?(\d{13,})",  # Matches "product/<13+ digit product_id>"
        "luisa_via_roma": r"/p/[^/]+/[^/]+/([^?]+)",  # Matches "/p/<product_id>/women"
    }
    if url:
        match = re.search(patterns[pattern], url)
        if match:
            return match.group(1)
    return None

def df_process(old_df,pattern):
    df = old_df.copy()
    df['url'] = df['url'].apply(lambda x: list(x.values())[0])
    df['product_id'] = df['product_id'].apply(lambda x: product_id_extractor(x, pattern=pattern))
    df['main_title'] = df['main_title'].apply(lambda x: list(x.values())[0]).str.lower().str.strip()
    df['description'] = df['description'].apply(lambda x: list(x.values())[0]).str.lower().str.strip()
    df['material'] = df['details'].apply(lambda x: x['material'])
    df['brand'] = df['details'].apply(lambda x: x['brand']).str.lower().str.strip()
    df['color'] = df['color'].str.lower().str.strip()
    df["previous_price"].fillna(df["current_price"], inplace=True)
    df["discounted"] = df["current_price"] < df["previous_price"]

    #maxmara_brands = ['weekend max mara', 'max mara', 'accessori', 'sportmax', "'s max mara", 'studio', 'the cube',
    #                  'leisure', 'sfilata', 'atelier', 'occhiali', 'accessori runway']
    #df = df[df['brand'].isin([brand for brand in maxmara_brands])]
    #df = df.reset_index(drop=True)
    
    del df['details']
    
    return df

In [4]:
maxmara_expanded = expand_dataframe(maxmara_df)
netaporter_expanded = expand_dataframe(netaporter_df)
luisaviaroma_expanded = expand_dataframe(luisaviaroma_df)
maxmara = df_process(maxmara_expanded,'max_mara')
netaporter = df_process(netaporter_expanded,'net_a_porter')
luisaviaroma = df_process(luisaviaroma_expanded,'luisa_via_roma')
print("Max Mara Data:")
display(maxmara.head(1))
print(maxmara.info())
print(maxmara.isnull().sum())
print("\nNet-A-Porter Data:")
display(netaporter.head(1))
print(netaporter.info())
print(netaporter.isnull().sum())
print("\nLuisa Via Roma Data:")
display(luisaviaroma.head(1))
print(luisaviaroma.info())
print(luisaviaroma.isnull().sum())

Max Mara Data:


,url,product_id,ref,main_title,description,full_path,color,current_price,previous_price,available_sizes,all_sizes,images,outlet_color,excluweb_color,material,brand,discounted
0,[https://us.maxmara.com/p-1081065106001-oggeri...,1081065106001,1081065106,cropped teddy bear icon coat in alpaca and wool,cropped jacket in the iconic teddy fabric of a...,"[[Homepage, Coats and Jackets, Coats and Jacke...",white,2890.0,2890.0,"[4, 6, 12, 0, 8, 2, 10]","[4, 6, 12, 0, 16, 8, 10, 2, 14]",[https://b2c-media.maxmara.com/sys-master/m0/M...,False,False,"Jersey fabric 62% alpaca, 26% virgin wool, 12%...",max mara,False


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3646 entries, 0 to 3645
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   url              3646 non-null   object 
 1   product_id       3646 non-null   object 
 2   ref              3646 non-null   int64  
 3   main_title       3646 non-null   object 
 4   description      3646 non-null   object 
 5   full_path        3646 non-null   object 
 6   color            3646 non-null   object 
 7   current_price    3646 non-null   float64
 8   previous_price   3646 non-null   float64
 9   available_sizes  3646 non-null   object 
 10  all_sizes        3646 non-null   object 
 11  images           3646 non-null   object 
 12  outlet_color     3646 non-null   bool   
 13  excluweb_color   3646 non-null   bool   
 14  material         3646 non-null   object 
 15  brand            3646 non-null   object 
 16  discounted       3646 non-null   bool   
dtypes: bool(3), fl

,url,product_id,ref,main_title,description,full_path,color,current_price,previous_price,available_sizes,all_sizes,images,outlet_color,excluweb_color,material,brand,discounted
0,[https://www.net-a-porter.com/en-us/shop/produ...,1647597332512685,1647597332512685,le 37 leather-trimmed raffia bucket bag,saint laurent's 'le 37' tote is the label's ch...,[[SAINT LAURENT]],beige,3200.0,3200.0,[ONE_SIZE],[ONE_SIZE],[https://www.net-a-porter.com/variants/images/...,False,False,"Raffia, brown leather (Calf) Hookfastening ta...",saint laurent,False


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7791 entries, 0 to 7790
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   url              7791 non-null   object 
 1   product_id       7788 non-null   object 
 2   ref              7791 non-null   int64  
 3   main_title       7791 non-null   object 
 4   description      7791 non-null   object 
 5   full_path        7791 non-null   object 
 6   color            7791 non-null   object 
 7   current_price    7791 non-null   float64
 8   previous_price   7791 non-null   float64
 9   available_sizes  7791 non-null   object 
 10  all_sizes        7791 non-null   object 
 11  images           7791 non-null   object 
 12  outlet_color     7791 non-null   bool   
 13  excluweb_color   7791 non-null   bool   
 14  material         7791 non-null   object 
 15  brand            7791 non-null   object 
 16  discounted       7791 non-null   bool   
dtypes: bool(3), fl

,url,product_id,ref,main_title,description,full_path,color,current_price,previous_price,available_sizes,all_sizes,images,outlet_color,excluweb_color,material,brand,discounted
0,[https://www.luisaviaroma.com/en-us/p/courrege...,80I-YZ0023,80I-YZ0023,rib knit viscose tank top,fixed spaghetti straps front logo detail model...,"[[women, COURREGES]]",black_9999,274.0,274.0,"[XS, S, M, L, XL]","[M, XS, XL, S, L]",[https://images.lvrcdn.com/c/2024/06/28/63ee90...,False,False,"70% Viscose, 30% Polyester",courreges,False


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1445 entries, 0 to 1444
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   url              1445 non-null   object 
 1   product_id       1445 non-null   object 
 2   ref              1445 non-null   object 
 3   main_title       1445 non-null   object 
 4   description      1445 non-null   object 
 5   full_path        1445 non-null   object 
 6   color            1445 non-null   object 
 7   current_price    1445 non-null   float64
 8   previous_price   1445 non-null   float64
 9   available_sizes  1445 non-null   object 
 10  all_sizes        1445 non-null   object 
 11  images           1445 non-null   object 
 12  outlet_color     1445 non-null   bool   
 13  excluweb_color   1445 non-null   bool   
 14  material         1445 non-null   object 
 15  brand            1445 non-null   object 
 16  discounted       1445 non-null   bool   
dtypes: bool(3), fl

In [5]:
maxmara_brands = list(maxmara['brand'].unique())
netaporter_filtered = netaporter[netaporter['brand'].isin(maxmara_brands)]
netaporter_filtered = netaporter_filtered.reset_index(drop=True)
luisaviaroma_filtered = luisaviaroma[luisaviaroma['brand'].isin(maxmara_brands)]
luisaviaroma_filtered = luisaviaroma_filtered.reset_index(drop=True)
available_brands = list(netaporter_filtered['brand'].value_counts().keys()) + list(luisaviaroma_filtered['brand'].value_counts().keys())
maxmara_filtered = maxmara[maxmara['brand'].isin(available_brands)]

In [6]:
maxmara_filtered.to_csv('maxmara_cleaned.csv',index=False)
netaporter_filtered.to_csv('netaporter_cleaned.csv',index=False)
luisaviaroma_filtered.to_csv('luisaviaroma_cleaned.csv',index=False)